# Classification: KNeighbors and Logistic regression

In [1]:
import sys
sys.path.append('..')

In [2]:
from source.code.data_loader import DataLoader
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

## Initial data loading

In [3]:
with open('../data/description/datasets', 'r') as source:
    datasets = source.readlines()
datasets = [dataset.replace('\n', '') for dataset in datasets]
dataset_path_pattern = '../data/{}/{}.{}.txt'
data_paths = [dataset_path_pattern.format('train', dataset, 'data') for dataset in datasets] + [dataset_path_pattern.format('test', dataset, 'test') for dataset in datasets]
data_loader = DataLoader(data_paths, '../data/description/columns', '../data/description/classes')
data_frame = data_loader.load()

# Dataset description

## EDA

In [4]:
data_frame.head(30)

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,...,TT4 measured,TT4,T4U measured,T4U,FTI measured,FTI,TBG measured,TBG,referral source,diagnosis
0,41.0,1.0,False,False,False,False,False,False,False,False,...,True,125.0,True,1.14,True,109.0,False,None,SVHC,False
1,23.0,1.0,False,False,False,False,False,False,False,False,...,True,102.0,False,NaN,False,NaN,False,None,other,False
2,46.0,0.0,False,False,False,False,False,False,False,False,...,True,109.0,True,0.91,True,120.0,False,None,other,False
3,70.0,1.0,True,False,False,False,False,False,False,False,...,True,175.0,False,NaN,False,NaN,False,None,other,False
4,70.0,1.0,False,False,False,False,False,False,False,False,...,True,61.0,True,0.87,True,70.0,False,None,SVI,False
5,18.0,1.0,True,False,False,False,False,False,False,False,...,True,183.0,True,1.30,True,141.0,False,None,other,False
6,59.0,1.0,False,False,False,False,False,False,False,False,...,True,72.0,True,0.92,True,78.0,False,None,other,False
7,80.0,1.0,False,False,False,False,False,False,False,False,...,True,80.0,True,0.70,True,115.0,False,None,SVI,True
8,66.0,1.0,False,False,False,False,False,False,False,False,...,True,123.0,True,0.93,True,132.0,False,None,SVI,False
9,68.0,0.0,False,False,False,False,False,False,False,False,...,True,83.0,True,0.89,True,93.0,False,None,SVI,False


In [5]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22632 entries, 0 to 971
Data columns (total 30 columns):
age                          22626 non-null float64
sex                          21732 non-null float64
on thyroxine                 22632 non-null bool
query on thyroxine           22632 non-null bool
on antithyroid medication    22632 non-null bool
sick                         22632 non-null bool
pregnant                     22632 non-null bool
thyroid surgery              22632 non-null bool
I131 treatment               22632 non-null bool
query hypothyroid            22632 non-null bool
query hyperthyroid           22632 non-null bool
lithium                      22632 non-null bool
goitre                       22632 non-null bool
tumor                        22632 non-null bool
hypopituitary                22632 non-null bool
psych                        22632 non-null bool
TSH measured                 22632 non-null bool
TSH                          20418 non-null float64
T3

После объединения могло возникнуть некоторое количество дупликатов, попробуем от них избавиться:

In [6]:
data_frame.drop_duplicates(inplace=True)

In [7]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4619 entries, 0 to 909
Data columns (total 30 columns):
age                          4618 non-null float64
sex                          4435 non-null float64
on thyroxine                 4619 non-null bool
query on thyroxine           4619 non-null bool
on antithyroid medication    4619 non-null bool
sick                         4619 non-null bool
pregnant                     4619 non-null bool
thyroid surgery              4619 non-null bool
I131 treatment               4619 non-null bool
query hypothyroid            4619 non-null bool
query hyperthyroid           4619 non-null bool
lithium                      4619 non-null bool
goitre                       4619 non-null bool
tumor                        4619 non-null bool
hypopituitary                4619 non-null bool
psych                        4619 non-null bool
TSH measured                 4619 non-null bool
TSH                          4286 non-null float64
T3 measured          

Судя по всему подавляющее большинство информации в датастах было дуплицировано.

Из описания видно, что признак TBG не содержит ни одного ненулевого значения, соответственно значение данного признакане было измерено ни для одного пациента. Данный признак (а соответственно и признак TBG measured) можно удалить из выборки:

In [8]:
data_frame.drop(['TBG measured', 'TBG'], axis='columns', inplace=True)

In [9]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4619 entries, 0 to 909
Data columns (total 28 columns):
age                          4618 non-null float64
sex                          4435 non-null float64
on thyroxine                 4619 non-null bool
query on thyroxine           4619 non-null bool
on antithyroid medication    4619 non-null bool
sick                         4619 non-null bool
pregnant                     4619 non-null bool
thyroid surgery              4619 non-null bool
I131 treatment               4619 non-null bool
query hypothyroid            4619 non-null bool
query hyperthyroid           4619 non-null bool
lithium                      4619 non-null bool
goitre                       4619 non-null bool
tumor                        4619 non-null bool
hypopituitary                4619 non-null bool
psych                        4619 non-null bool
TSH measured                 4619 non-null bool
TSH                          4286 non-null float64
T3 measured          

Заметим, что TSH measured, T3 measured, TT4 measured, T4U measured и FTI measured просто показывают, было ли измерено у пациента значение признаков TSH, T3, TT4, T4U и FTI соответственно. Данная информация избыточна, поскольку по значению этих признаков можно понять были они измерены или нет (наличие или отсутствие NaN), TSH measured, T3 measured, TT4 measured, T4U measured и FTI measured также можно удалить из выборки:

In [10]:
data_frame.drop(['TSH measured', 'T3 measured', 'TT4 measured', 'T4U measured', 'FTI measured'], axis='columns', inplace=True)

In [11]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4619 entries, 0 to 909
Data columns (total 23 columns):
age                          4618 non-null float64
sex                          4435 non-null float64
on thyroxine                 4619 non-null bool
query on thyroxine           4619 non-null bool
on antithyroid medication    4619 non-null bool
sick                         4619 non-null bool
pregnant                     4619 non-null bool
thyroid surgery              4619 non-null bool
I131 treatment               4619 non-null bool
query hypothyroid            4619 non-null bool
query hyperthyroid           4619 non-null bool
lithium                      4619 non-null bool
goitre                       4619 non-null bool
tumor                        4619 non-null bool
hypopituitary                4619 non-null bool
psych                        4619 non-null bool
TSH                          4286 non-null float64
T3                           3778 non-null float64
TT4               

Заметим, что в данных присутствуют записи о неопределившихся с полом:

In [12]:
data_frame[(data_frame.sex.isnull())].sort_index().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184 entries, 16 to 2786
Data columns (total 23 columns):
age                          184 non-null float64
sex                          0 non-null float64
on thyroxine                 184 non-null bool
query on thyroxine           184 non-null bool
on antithyroid medication    184 non-null bool
sick                         184 non-null bool
pregnant                     184 non-null bool
thyroid surgery              184 non-null bool
I131 treatment               184 non-null bool
query hypothyroid            184 non-null bool
query hyperthyroid           184 non-null bool
lithium                      184 non-null bool
goitre                       184 non-null bool
tumor                        184 non-null bool
hypopituitary                184 non-null bool
psych                        184 non-null bool
TSH                          165 non-null float64
T3                           156 non-null float64
TT4                          169 non-

Глянем, есть ли среди неопределившихся с полом беременные:

In [13]:
data_frame[(data_frame.sex.isnull()) & (data_frame.pregnant)]

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,...,tumor,hypopituitary,psych,TSH,T3,TT4,T4U,FTI,referral source,diagnosis
1609,73.0,NaN,False,False,False,False,True,False,False,False,...,False,False,False,2.2,2.5,110.0,1.28,85.0,other,False
471,21.0,NaN,False,False,False,False,True,False,False,False,...,True,False,False,2.4,3.5,171.0,1.49,115.0,STMW,False
471,21.0,NaN,False,False,False,False,True,False,False,False,...,True,False,False,2.4,3.5,171.0,1.49,115.0,STMW,True


Поскольку пол - довольно важный признак и поскольку неопределившиеся составляют $100 \cdot \frac{185}{4680} \approx 4\%$, будет проще избавиться от данной части выборки:

In [14]:
data_frame.drop(data_frame[(data_frame.sex.isnull())].index, inplace=True)

In [15]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4341 entries, 0 to 909
Data columns (total 23 columns):
age                          4340 non-null float64
sex                          4341 non-null float64
on thyroxine                 4341 non-null bool
query on thyroxine           4341 non-null bool
on antithyroid medication    4341 non-null bool
sick                         4341 non-null bool
pregnant                     4341 non-null bool
thyroid surgery              4341 non-null bool
I131 treatment               4341 non-null bool
query hypothyroid            4341 non-null bool
query hyperthyroid           4341 non-null bool
lithium                      4341 non-null bool
goitre                       4341 non-null bool
tumor                        4341 non-null bool
hypopituitary                4341 non-null bool
psych                        4341 non-null bool
TSH                          4032 non-null float64
T3                           3538 non-null float64
TT4               

Остался еще один персонаж, у которого не указан возраст:

In [16]:
data_frame[(data_frame.age.isnull())]

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,...,tumor,hypopituitary,psych,TSH,T3,TT4,T4U,FTI,referral source,diagnosis
1985,NaN,1.0,True,False,False,False,False,False,True,False,...,False,False,False,0.6,1.5,120.0,0.82,146.0,other,False


Можно было бы посчитать сколько-нибудь ближайших к данному персонажу личностей потом по медиане взять возраст,
однако для этого нужно либо ранжировать всю выборку либо случайным образом доставать похожих по симптомам пациентов.
Гораздо проще исключить пока этот объект из выборки, поскольку вряд ли отсутствие одного пациента существенно сместит результаты обучения:

In [17]:
data_frame.drop(data_frame[(data_frame.age.isnull())].index, inplace=True)

In [18]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4340 entries, 0 to 909
Data columns (total 23 columns):
age                          4340 non-null float64
sex                          4340 non-null float64
on thyroxine                 4340 non-null bool
query on thyroxine           4340 non-null bool
on antithyroid medication    4340 non-null bool
sick                         4340 non-null bool
pregnant                     4340 non-null bool
thyroid surgery              4340 non-null bool
I131 treatment               4340 non-null bool
query hypothyroid            4340 non-null bool
query hyperthyroid           4340 non-null bool
lithium                      4340 non-null bool
goitre                       4340 non-null bool
tumor                        4340 non-null bool
hypopituitary                4340 non-null bool
psych                        4340 non-null bool
TSH                          4031 non-null float64
T3                           3537 non-null float64
TT4               

**TODO:** извлечь referral source из выборки

Отделим целевой признак (диагноз) от предикторов (первых 21 признаков):

In [19]:
y = data_frame['diagnosis']
X = data_frame.drop(['diagnosis'], axis='columns')

Теперь заметим, что в последних пяти признаках есть пропуски.
Собственно, это те пациенты, для которых показания по соответствующим признакам не были измерены.
Поскольку в используемых алгоритмах так или иначе используется суммарный вклад признаков, было бы осмысленно в качестве заполнения использовать медианы
(поскольку медиана занимает серединное положение, она не будет давать сильного смещения для объектов, которые по остальным признакам близки друг к другу, таким образом снижается шанс напороться на false-positive.
С другой стороны медиана не сместится в ситуации, когда значение признака аномально высоко/низко).

In [ ]:
x_columns = X.columns

In [ ]:
imputer = Imputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(X), columns=x_columns)

In [ ]:
X.head(30)

In [ ]:
X.info()

Итак, у нас есть некий базовый слепок выборки, теперь попробуем обучить логистическую регрессию и метод К-ближайших соседей.

Да, вот так вслепую, без особого вмешательства, просто чтобы глянуть, что получится, если не предпринимать никаких шагов.

Разделим предварительно выборку на обучающую и тестовую:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
k_n_classifier = KNeighborsClassifier()
k_n_classifier.fit(X_train, y_train)

In [ ]:
l_r_classifier = LogisticRegression()
l_r_classifier.fit(X_train, y_train)

In [ ]:
k_n_y_pred = k_n_classifier.predict(X_test)
l_r_y_pred = l_r_classifier.predict(X_test)

In [ ]:
k_n_precision_sc = precision_score(y_test, k_n_y_pred)
k_n_accuracy_sc = accuracy_score(y_test, k_n_y_pred)
k_n_recall_sc = recall_score(y_test, k_n_y_pred)
k_n_conf_matrix = confusion_matrix(y_test, k_n_y_pred)

In [ ]:
l_r_precision_sc = precision_score(y_test, l_r_y_pred)
l_r_accuracy_sc = accuracy_score(y_test, l_r_y_pred)
l_r_recall_sc = recall_score(y_test, l_r_y_pred)
l_r_conf_matrix = confusion_matrix(y_test, l_r_y_pred)

In [ ]:
sns.heatmap(k_n_conf_matrix, annot=True, fmt="d")
plt.show()

In [ ]:
sns.heatmap(l_r_conf_matrix, annot=True, fmt="d")
plt.show()

In [ ]:
print("K-nearest neighbours | Precision: {0:f}, Recall: {1:f}, Accuracy: {2:f}".format(k_n_precision_sc, k_n_recall_sc, k_n_accuracy_sc))

In [ ]:
print("Logistic regression | Precision: {0:f}, Recall: {1:f}, Accuracy: {2:f}".format(l_r_precision_sc, l_r_recall_sc, l_r_accuracy_sc))

Собственно, ничего особо хорошего не получилось.

Что в первом, что во втором случае значения Precision и Recall очень низкие.

Глянем на распределение классов в обучающей и контрольной выборке:

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()